In [4]:
# Import initial dependencies
import pandas as pd
import numpy as np

In [5]:
# Convert CSVs to DFs
features = "../../prediction_modeling/output/reduced_wine_features.csv"
targets = "../../prediction_modeling/output/reduced_wine_targets.csv"

features_df = pd.read_csv(features)
targets_df =pd.read_csv(targets) 

In [6]:
# Check feature DF head
features_df.head()

,price,white,country_Argentina,country_Australia,country_Chile,country_Germany,country_Portugal,country_Spain,country_US,"flavor_categories_taste_notes_light, fruity","flavor_categories_taste_notes_medium bodied, balanced"
0,15.0,0.0,0,0,0,0,1,0,0,1,0
1,65.0,0.0,0,0,0,0,0,0,1,0,1
2,19.0,0.0,0,0,0,0,0,0,1,1,0
3,34.0,0.0,0,0,0,0,0,0,1,0,1
4,30.0,0.0,1,0,0,0,0,0,0,0,1


In [7]:
# Check feature DF shape
features_df.shape

(109662, 11)

In [9]:
# Check targets DF
targets_df.head()

,points
0,87
1,87
2,87
3,87
4,87


In [10]:
# Check Dtypes
targets_df.dtypes

points    int64
dtype: object

In [12]:
# Check feature and target DF shape
print(features_df.shape, targets_df.shape)

(109662, 11) (109662, 1)


In [13]:
# Assign the features and target to X and y abd check assignments
raw_feature_data = features_df.values
raw_target_data = targets_df.values
X = raw_feature_data[:, 0:12]
y = raw_target_data.reshape(-1,1)

print(X, y)

[[15.  0.  0. ...  0.  1.  0.]
 [65.  0.  0. ...  1.  0.  1.]
 [19.  0.  0. ...  1.  1.  0.]
 ...
 [70.  0.  0. ...  0.  0.  0.]
 [25.  0.  0. ...  1.  0.  0.]
 [38.  0.  0. ...  0.  0.  0.]] [[87]
 [87]
 [87]
 ...
 [90]
 [90]
 [91]]


# SPLIT, NORMALIZE, AND ENCODE THE DATA

In [15]:
# Create the train and test sets for the features and target
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [16]:
# Scale the features using the MinMax scaler since we know their values
from sklearn.preprocessing import LabelEncoder, MinMaxScaler

X_scaler = MinMaxScaler().fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

y_scaler = MinMaxScaler().fit(y_train)
y_train_scaled = y_scaler.transform(y_train)
y_test_scaled = y_scaler.transform(y_test)

# HYPERPERAMETER TUNING

In [18]:
# Import the sequential and dense modules fo build my NN
from sklearn.model_selection import GridSearchCV
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.wrappers.scikit_learn import KerasRegressor
from keras.constraints import maxnorm

# RUN XGB REGRESSOR ON SCALED DATA AND INSPECT THE RESULTS

In [20]:
print('XGB Regressor...')

# Import dependencies
from xgboost import XGBRegressor
from datetime import datetime
import pprint
import os

# Create start time and model for XGB Regressor
start_time  = datetime.now()
xgb_reg = XGBRegressor() 

# Create parameter dictionary and insert into the grid search constructor
parameters = {'n_estimators': [200, 300, 500], 'max_depth':[3,5,7,9], \
              'learning_rate':[.01, .001, .0001], 'batch_size':[10, 20, 40, 60, 80, 100]}

grid_search = GridSearchCV(estimator=xgb_reg, param_grid=parameters, cv=10, n_jobs=-1)


# Print out parameters
print("parameters:")
pprint.pprint(parameters)


# Fit model
grid_search.fit(X_train_scaled, y_train_scaled.ravel())


# Print best sore, best parameters
print("Best score: %0.3f" % grid_search.best_score_)
print("Best parameters set:")
best_parameters=grid_search.best_estimator_.get_params()


# Loop through params and get keys + print time cost
for param_name in sorted(parameters.keys()):
    print("\t%s: %r" % (param_name, best_parameters[param_name]))
end_time = datetime.now()
print(f'Select Done..., Time Cost: {((end_time - start_time).seconds)}.') 

XGB Regressor...
parameters:
{'batch_size': [10, 20, 40, 60, 80, 100],
 'learning_rate': [0.01, 0.001, 0.0001],
 'max_depth': [3, 5, 7, 9],
 'n_estimators': [200, 300, 500]}
[04:51:40] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
Best score: 0.415
Best parameters set:
	batch_size: 10
	learning_rate: 0.01
	max_depth: 7
	n_estimators: 500
Select Done..., Time Cost: 8505.


In [21]:
from sklearn.metrics import mean_squared_error, r2_score

# Use our model to predict a value
predicted = grid_search.predict(X_test_scaled)

# Score the prediction with mse and r2
mse = mean_squared_error(y_test_scaled, predicted)
r2 = r2_score(y_test_scaled, predicted)

# Print scores
print(f"Mean Squared Error (MSE): {mse}")
print(f"R-squared (R2): {r2}")

Mean Squared Error (MSE): 0.013813830501431295
R-squared (R2): 0.41036356954076403


In [22]:
# Create list of predictions and check
predicted_list = grid_search.predict(X_test_scaled)

predicted_list[:10]

array([0.42429772, 0.30697387, 0.33906835, 0.39895833, 0.56125796,
       0.399701  , 0.47776294, 0.5480776 , 0.40939444, 0.45089942],
      dtype=float32)

In [23]:
# Create list of actual values and check
actual_list = y_test_scaled

actual_list[:10]

array([[0.55],
       [0.45],
       [0.45],
       [0.4 ],
       [0.4 ],
       [0.65],
       [0.4 ],
       [0.6 ],
       [0.25],
       [0.35]])

In [24]:
# Create DF to further compare predictions with actual points
prediction_comparrisons_df = pd.DataFrame({'XGB_Predictions':predicted_list,
                                           'Actual_Points':actual_list.ravel()
                                          })

prediction_comparrisons_df.head(10)

,XGB_Predictions,Actual_Points
0,0.424298,0.55
1,0.306974,0.45
2,0.339068,0.45
3,0.398958,0.40
4,0.561258,0.40
5,0.399701,0.65
6,0.477763,0.40
7,0.548078,0.60
8,0.409394,0.25
9,0.450899,0.35


In [25]:
# Export prediction comparrisons
prediction_comparrisons_df.to_csv(r'C:\Users\howar\Desktop\Data Science Boot Camp\Group_Project_3\prediction_modeling\output\predictions_vs_acttual.csv', index=False)

In [60]:
# Save the tuned model for future use
import joblib
filename = 'saved_models\XGB_Regressor_scaled.sav'
joblib.dump(grid_search, filename)

['saved_models\\XGB_Regressor_scaled.sav']

# Run XGB Regressor with unscalled data and recommended parameters from the previous test

In [49]:
print('Select Model...')
from xgboost import XGBRegressor
from datetime import datetime
import pprint
import os

start_time_2  = datetime.now()
xgb_reg_2 = XGBRegressor() 


parameters_reg_2 = {'n_estimators': [850], 'max_depth':[7], \
              'learning_rate':[.01], 'batch_size':[10]}

grid_search_2 = GridSearchCV(estimator=xgb_reg_2, param_grid=parameters_reg_2, cv=10, n_jobs=-1)


print("parameters:")
pprint.pprint(parameters_reg_2)


grid_2 = grid_search_2.fit(X_train, y_train.ravel())


print("Best score: %0.3f" % grid_2.best_score_)
print("Best parameters set:")
best_parameters_2=grid_2.best_estimator_.get_params()


for param_name in sorted(parameters_reg_2.keys()):
    print("\t%s: %r" % (param_name, best_parameters_2[param_name]))
end_time_2 = datetime.now()
print(f'Select Done..., Time Cost: {((end_time_2 - start_time_2).seconds)}.') 

Select Model...
parameters:
{'batch_size': [10],
 'learning_rate': [0.01],
 'max_depth': [7],
 'n_estimators': [850]}
[07:38:27] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
Best score: 0.415
Best parameters set:
	batch_size: 10
	learning_rate: 0.01
	max_depth: 7
	n_estimators: 850
Select Done..., Time Cost: 153.


In [50]:
from sklearn.metrics import mean_squared_error, r2_score

# Use our model to predict a value
predicted_2 = grid_2.predict(X_test)

# Score the prediction with mse and r2
mse_2 = mean_squared_error(y_test, predicted_2)
r2_2 = r2_score(y_test, predicted_2)

# Print scores
print(f"Mean Squared Error (MSE): {mse_2}")
print(f"R-squared (R2): {r2_2}")

Mean Squared Error (MSE): 5.517620949521715
R-squared (R2): 0.4112077890042899


In [ ]:
# Create list of predictions and check
predicted_list_unscaled = grid_2.predict(X_test)

predicted_list_unscaled

In [52]:
# Create list of actual values and check
actual_list_unscaled = y_test

actual_list_unscaled

array([[91],
       [89],
       [89],
       ...,
       [91],
       [96],
       [90]], dtype=int64)

In [56]:
# Create DF for unscaled test to further compare predictions with actual points
prediction_comparrisons_unscaled_df = pd.DataFrame({'XGB_Predictions (unscaled)' : predicted_list_unscaled,
                                           'Actual_Points (unscaled)' : actual_list_unscaled.ravel()
                                          })

prediction_comparrisons_unscaled_df.head(40)

,XGB_Predictions (unscaled),Actual_Points (unscaled)
0,88.490555,91
1,86.018265,89
2,86.880157,89
3,88.046768,88
4,91.225143,88
5,88.015190,93
6,89.570747,88
7,90.953506,92
8,88.181931,85
9,88.933014,87


In [57]:
# Export unscaled results
prediction_comparrisons_unscaled_df.to_csv(r'C:\Users\howar\Desktop\Data Science Boot Camp\Group_Project_3\prediction_modeling\output\predictions_vs_actual_unscaled.csv', index=False)

In [62]:
# Save the tuned model for future use
import joblib
filename = 'saved_models\XGB_Regressor_unscaled_(BEST).sav'
joblib.dump(grid_2, filename)

['saved_models\\XGB_Regressor_unscaled_(BEST).sav']

# SEE WHAT A CLASSIFIER DOES WITH THE DATA

In [65]:
from sklearn.linear_model import Ridge
from sklearn.model_selection import GridSearchCV

# Test other models for comparrison
# NOTE, THESE ARE CLASSIFICATION MODELS I AM TRYIN OUT JUST TO SEE HOW IT PERFORMS ON THE DATA
params_clf={'alpha': [25,10,4,2,1.0,0.8,0.5,0.3,0.2,0.1,0.05,0.02,0.01],
            'normalize': [True, False]}


# Creat, fit, and test model
rdg_clf = Ridge()
clf_grid = GridSearchCV(rdg_clf, params_clf, cv=2, verbose = 1, scoring = 'neg_mean_squared_error')
clf_grid.fit(X_train_scaled, y_train_scaled)

Fitting 2 folds for each of 26 candidates, totalling 52 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  52 out of  52 | elapsed:    0.5s finished


GridSearchCV(cv=2, error_score='raise-deprecating',
             estimator=Ridge(alpha=1.0, copy_X=True, fit_intercept=True,
                             max_iter=None, normalize=False, random_state=None,
                             solver='auto', tol=0.001),
             iid='warn', n_jobs=None,
             param_grid={'alpha': [25, 10, 4, 2, 1.0, 0.8, 0.5, 0.3, 0.2, 0.1,
                                   0.05, 0.02, 0.01],
                         'normalize': [True, False]},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring='neg_mean_squared_error', verbose=1)

In [67]:
# Print the score and best params for the activation model
print("Best: %f using %s" % (clf_grid.best_score_, clf_grid.best_params_))

Best: -0.018771 using {'alpha': 0.02, 'normalize': False}


In [68]:
from sklearn.metrics import mean_squared_error, r2_score

# Use our model to predict a value
predicted_3 = clf_grid.predict(X_test_scaled)

# Score the prediction with mse and r2
mse_3 = mean_squared_error(y_test_scaled, predicted_3)
r2_3 = r2_score(y_test_scaled, predicted_3)

# Print results
print(f"Mean Squared Error (MSE): {mse_3}")
print(f"R-squared (R2): {r2_3}")

Mean Squared Error (MSE): 0.018561234926338842
R-squared (R2): 0.20772299140721662


In [69]:
# Create list of predictions and check
predicted_list_clf = clf_grid.predict(X_test)

predicted_list_clf

array([[149.36928326],
       [ 60.91139232],
       [112.09340453],
       ...,
       [107.52408247],
       [382.10829152],
       [ 84.17701498]])

In [70]:
# Create list of actual values and check
actual_list_clf = y_test

actual_list_clf

array([[91],
       [89],
       [89],
       ...,
       [91],
       [96],
       [90]], dtype=int64)

In [71]:
# Save the tuned model for future use
import joblib
filename = 'saved_models\Ridge_Classifier.sav'
joblib.dump(clf_grid, filename)

['saved_models\\Ridge_Classifier.sav']

# TESTING OUT OTHER MODEL EXAMPLES I FOUND

In [72]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.model_selection import cross_val_score

# Try some different model examples I found when doing my research...
def cross_validate_best_known():
    '''
        import and clean the wine data, then do a corss validation on each of the three models we are
        training here. A RandomForest, a GradientBoost, and an AdaBoost backed by a DecisionTree. Print
        the scores.

        The parameters we're using here are the "best" that we've found so far using a grid search.
    '''

rf = RandomForestRegressor(max_features=2, min_samples_split=4, n_estimators=50, min_samples_leaf=2)
gb = GradientBoostingRegressor(loss='quantile', learning_rate=0.0001, n_estimators=50, max_features='log2', min_samples_split=2, max_depth=1)
ada_tree_backing = DecisionTreeRegressor(max_features='sqrt', splitter='random', min_samples_split=4, max_depth=3)
ab = AdaBoostRegressor(ada_tree_backing, learning_rate=0.0001, loss='square', n_estimators=100000)

# Fit the data to the Random Forest Regressor
rf.fit(X_train_scaled, y_train_scaled.ravel())


# validate.cross_v_scores([rf, gb, ab], X_train_scaled, y_train_scaled)
# # RandomForestRegressor -- RMLSE: -0.596797712098, R2: 0.0272065373946
# GradientBoostingRegressor -- RMLSE: -0.996134592541, R2: -2.37202164829
# AdaBoostRegressor -- RMLSE: -0.706385708459, R2: -0.103966980393 

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
                      max_features=2, max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=2, min_samples_split=4,
                      min_weight_fraction_leaf=0.0, n_estimators=50,
                      n_jobs=None, oob_score=False, random_state=None,
                      verbose=0, warm_start=False)

In [ ]:
# Use our model to predict a value
predicted_rf = rf.predict(X_test_scaled)

# Score the prediction with mse and r2
mse_rf = mean_squared_error(y_test_scaled, predicted_rf)
r2_rf = r2_score(y_test_scaled, predicted_rf)

# Print results
print(f"Mean Squared Error (MSE): {mse_rf}")
print(f"R-squared (R2): {r2_rf}")

In [ ]:
# Create list of predictions and check
predicted_list_rf = rf.predict(X_test_scaled)

predicted_list_rf

In [ ]:
# Create list of actual values and check
actual_list_rf = y_test_scaled

actual_list_rf

In [ ]:
# Save the tuned model for future use
import joblib
filename = 'saved_models\RF_Regr.sav'
joblib.dump(rf, filename)

In [73]:
# Fit the data to the Gradient Boosting Regressor
gb.fit(X_train_scaled, y_train_scaled.ravel())

GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
                          learning_rate=0.0001, loss='quantile', max_depth=1,
                          max_features='log2', max_leaf_nodes=None,
                          min_impurity_decrease=0.0, min_impurity_split=None,
                          min_samples_leaf=1, min_samples_split=2,
                          min_weight_fraction_leaf=0.0, n_estimators=50,
                          n_iter_no_change=None, presort='auto',
                          random_state=None, subsample=1.0, tol=0.0001,
                          validation_fraction=0.1, verbose=0, warm_start=False)

In [74]:
# Use our model to predict a value
predicted_4 = gb.predict(X_test_scaled)

# Score the prediction with mse and r2
mse_4 = mean_squared_error(y_test_scaled, predicted_4)
r2_4 = r2_score(y_test_scaled, predicted_4)

# Print results
print(f"Mean Squared Error (MSE): {mse_4}")
print(f"R-squared (R2): {r2_4}")

Mean Squared Error (MSE): 0.07489888217478236
R-squared (R2): -2.1970212408751997


In [ ]:
# Create list of predictions and check
predicted_list_gb = gb.predict(X_test_scaled)

predicted_list_gb

In [ ]:
# Create list of actual values and check
actual_list_gb = y_test_scaled

actual_list_gb

In [ ]:
# Save the tuned model for future use
import joblib
filename = 'saved_models\GB_Regr.sav'
joblib.dump(gb, filename)

In [75]:
# Fit the data to the ADA Tree Backing Regressor
ada_tree_backing.fit(X_train_scaled, y_train_scaled.ravel())

DecisionTreeRegressor(criterion='mse', max_depth=3, max_features='sqrt',
                      max_leaf_nodes=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=4, min_weight_fraction_leaf=0.0,
                      presort=False, random_state=None, splitter='random')

In [76]:
# Use our model to predict a value
predicted_5 = ada_tree_backing.predict(X_test_scaled)

# Score the prediction with mse and r2
mse_5 = mean_squared_error(y_test_scaled, predicted_5)
r2_5 = r2_score(y_test_scaled, predicted_5)

# Print Results
print(f"Mean Squared Error (MSE): {mse_5}")
print(f"R-squared (R2): {r2_5}")

Mean Squared Error (MSE): 0.022553979373154898
R-squared (R2): 0.037294696148155615


In [ ]:
# Create list of predictions and check
predicted_list_ada_tree_backing = ada_tree_backing.predict(X_test_scaled)

predicted_list_ada_tree_backing

In [ ]:
# Create list of actual values and check
actual_list_ada_tree_backing = y_test_scaled

actual_list_ada_tree_backing

In [ ]:
# Save the tuned model for future use
import joblib
filename = 'saved_models\ADA_Tree_Backing.sav'
joblib.dump(ada_tree_backing, filename)

In [ ]:
# Fit the data to the ADA Boosting Regressor
ab.fit(X_train_scaled, y_train_scaled.ravel())

In [ ]:
# Use our model to predict a value
predicted_6 = ab.predict(X_test_scaled)

# Score the prediction with mse and r2
mse_6 = mean_squared_error(y_test_scaled, predicted_6)
r2_6 = r2_score(y_test_scaled, predicted_6)

# Print Results
print(f"Mean Squared Error (MSE): {mse_6}")
print(f"R-squared (R2): {r2_6}")

In [ ]:
# Create list of predictions and check
predicted_list_ab = ab.predict(X_test_scaled)

predicted_list_ab

In [ ]:
# Create list of actual values and check
actual_list_ab = y_test_scaled

actual_list_ab

In [ ]:
# Save the tuned model for future use
import joblib
filename = 'saved_models\AB_Reg.sav'
joblib.dump(ab, filename)

# TESTING MY BUILD OF THE KERAS REGRESSOR

In [ ]:
# Create the base Keras classifier wrapper for activation method
def create_base_model(activation='relu'):
    
    model = Sequential()
    model.add(Dense(units=1000, activation='relu', input_dim=11))
    model.add(Dense(units=1000, activation='relu'))
    model.add(Dense(units=1000, activation='relu'))
    model.add(Dense(units=1000, activation='relu'))
    model.add(Dense(units=1, activation='linear'))
    model.compile(loss='mean_squared_error', optimizer='adam')
    
    return model


# Create activation model
keras_model = KerasRegressor(build_fn=create_base_model, epochs=20, batch_size=88)


# Define the activation grid search parameters
batch_size = [10, 32, 64, 88, 100, 150]


# Assign the activation grid search parameters, fit, and run the model
keras_param_grid = dict(batch_size=batch_size)
keras_grid = GridSearchCV(estimator=keras_model, param_grid=keras_param_grid, n_jobs=-1)
keras_grid_result = keras_grid.fit(X_train_scaled, y_train_scaled)

In [ ]:
# Print the score and best params for the activation model
print("Best: %f using %s" % (keras_grid_result.best_score_, activation_grid_result.best_params_))

In [ ]:
print(f"Training Data Score: {keras_grid_result.score(X_train_scaled, y_train_scaled)}")
print(f"Testing Data Score: {keras_grid_result.score(X_train_scaled, y_train_scaled)}")

In [ ]:
# Use our model to predict a value
predicted_7 = keras_grid_result.predict(X_test_scaled)

# Score the prediction with mse and r2
mse_7 = mean_squared_error(y_test_scaled, predicted_7)
r2_7 = r2_score(y_test_scaled, predicted_7)

# Print Results
print(f"Mean Squared Error (MSE): {mse_7}")
print(f"R-squared (R2): {r2_7}")

In [ ]:
# Create list of predictions and check
predicted_list_keras = keras_grid_result.predict(X_test_scaled)

predicted_list_keras

In [ ]:
# Create list of actual values and check
actual_list_keras = y_test_scaled

actual_list_keras

In [ ]:
# Save the tuned model for future use
import joblib
filename = 'saved_models\Keras_Reg_NN.sav'
joblib.dump(keras_grid_result, filename)

In [ ]:
# # Create the base Keras classifier wrapper for the model
# def create_model(activation='adam', dropout_rate=0.0):
    
#     model = Sequential()
#     model.add(Dense(units=100, activation='relu', input_dim=5))
#     model.add(Dense(units=100, activation='relu'))
#     model.add(Dense(units=100, activation='tanh'))
#     model.add(Dense(units=100, activation='relu'))
#     model.add(Dense(units=3, activation='softmax'))
#     model.add(Dropout(dropout_rate))
#     model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    
#     return model


# # Create model
# model = KerasClassifier(build_fn=create_model, epochs=250, batch_size=88)


# # Define the activation and dropout rate grid search parameters
# activation = ['softmax', 'softplus', 'softsign', 'relu', 'tanh', 'sigmoid', 'hard_sigmoid', 'linear']
# dropout_rate = [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]


# # Assign the grid search parameters, fit, and run the model
# param_grid = dict(activation=activation, dropout_rate=dropout_rate)
# fit_grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1)
# grid_result = fit_grid.fit(X_train_scaled, y_train_categorical)

In [ ]:
# # Print the score and best params for the activation model
# print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

In [ ]:
# # Create the base Keras classifier wrapper for optimizer model
# def create_optimizer_model(optimizer='adam'):
    
#     model = Sequential()
#     model.add(Dense(units=100, activation='hard_sigmoid', input_dim=5))
#     model.add(Dense(units=100, activation='hard_sigmoid'))
#     model.add(Dense(units=100, activation='hard_sigmoid'))
#     model.add(Dense(units=100, activation='hard_sigmoid'))
#     model.add(Dense(units=3, activation='hard_sigmoid'))
#     model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    
#     return model


# # Create optimizer model
# optimizer_model = KerasClassifier(build_fn=create_optimizer_model, epochs=250, batch_size=88)


# # Define the optimizer grid search parameters
# optimizer = ['SGD', 'RMSprop', 'Adagrad', 'Adadelta', 'Adam', 'Adamax', 'Nadam']


# # Assign the optimizer grid search parameters, fit, and run the model
# optimizer_param_grid = dict(optimizer=optimizer)
# optimizer_grid = GridSearchCV(estimator=optimizer_model, param_grid=optimizer_param_grid, n_jobs=-1)
# optimizer_grid_result = optimizer_grid.fit(X_train_scaled, y_train_categorical)

In [ ]:
# # Print the score and best params for the optimizer model
# print("Best: %f using %s" % (optimizer_grid_result.best_score_, optimizer_grid_result.best_params_))

In [ ]:
# # Create the base Keras classifier wrapper for dropout model
# def create_dropout_model(dropout_rate=0.0):
    
#     model = Sequential()
#     model.add(Dense(units=100, activation='relu', input_dim=5))
#     model.add(Dense(units=100, activation='relu'))
#     model.add(Dense(units=100, activation='tanh'))
#     model.add(Dense(units=100, activation='relu'))
#     model.add(Dense(units=3, activation='softmax'))
#     model.add(Dropout(dropout_rate))
#     model.compile(loss='categorical_crossentropy', optimizer='Adadelta', metrics=['accuracy']) 
    
#     return model


# # Create the dropout model
# dropout_model = KerasClassifier(build_fn=create_dropout_model, epochs=250, batch_size=88)


# # Define the dropout grid search parameters
# dropout_rate = [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]


# # Assign the dropout grid search parameters, fit, and run the model
# dropout_param_grid = dict(dropout_rate=dropout_rate)
# dropout_grid = GridSearchCV(estimator=dropout_model, param_grid=dropout_param_grid, n_jobs=-1)
# dropout_grid_result = dropout_grid.fit(X_train_scaled, y_train_categorical)

In [ ]:
# # Print the score and best params for the dropout model
# print("Best: %f using %s" % (dropout_grid_result.best_score_, dropout_grid_result.best_params_))

In [ ]:
# print(f"Training Data Score: {model2.score(X_train_scaled, y_train)}")
# print(f"Testing Data Score: {model2.score(X_test_scaled, y_test)}")